In [1]:
! pip install flair

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 10.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 9.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install scikit-learn matplotlib seaborn

In [3]:
from typing import List, Tuple, Dict

def convert_to_flair_format(data: Dict, output_file: str):
    for annotation in data['annotations']:
        words = annotation[0].split()
        tags = ['O'] * len(words)

        for start, end, label in annotation[1]['entities']:
            start_word = len(annotation[0][:start].split())
            end_word = len(annotation[0][:end].split())

            tags[start_word] = 'B-' + label
            for i in range(start_word + 1, end_word):
                tags[i] = 'I-' + label

        for word, tag in zip(words, tags):
            output_file.write(f'{word} {tag}\n')

        output_file.write('\n')

In [3]:
import flair
from flair.data import Sentence
from typing import List, Tuple, Dict

def convert_to_flair_format(data: Dict, output_file: str):
    for annotation in data['annotations']:
        words = annotation[0].split()
        tags = ['O'] * len(words)

        for start, end, label in annotation[1]['entities']:
            start_word = len(annotation[0][:start].split())
            end_word = len(annotation[0][:end].split())

            tags[start_word] = 'B-' + label
            for i in range(start_word + 1, end_word):
                tags[i] = 'I-' + label

        count = 0
        for word, tag in zip(words, tags):
            output_file.write(f'{word} {tag}\n')
            count += 1
            if count % 10 == 0:
                if tag == 'O':
                    output_file.write('\n')
                else:
                    for j in range(count, len(tags)):
                        if tags[j] == 'O':
                            output_file.write(f'{words[j]} {tags[j]}\n')
                            output_file.write('\n')
                            count = j
                            break

        output_file.write('\n')

### Prepare Train Data

In [2]:
import os
import json

train_folder = 'data/dataset/train_annotations'
train_file = 'data/train.txt'

if not os.path.exists(os.path.dirname(train_file)):
    os.makedirs(os.path.dirname(train_file))

with open(train_file, 'w') as outfile:
    for filename in os.listdir(train_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(train_folder, filename)

            with open(json_path, 'r') as file:
                json_data = json.load(file)

            convert_to_flair_format(json_data, outfile)


### Prepare Test Data

In [3]:
import os
import json

test_folder = 'data/dataset/test_annotations'
test_file = 'data/test.txt'

if not os.path.exists(os.path.dirname(test_file)):
    os.makedirs(os.path.dirname(test_file))

with open(test_file, 'w') as outfile:
    for filename in os.listdir(test_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(test_folder, filename)

            with open(json_path, 'r') as file:
                json_data = json.load(file)

            convert_to_flair_format(json_data, outfile)


### Load Train and Test Data

In [1]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
import flair
from flair.data import Sentence

# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'data'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt')

/home/dragon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dragon/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dragon/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dragon/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._regist

2024-03-09 11:50:51,902 Reading data from data
2024-03-09 11:50:51,902 Train: data/train.txt
2024-03-09 11:50:51,902 Dev: None
2024-03-09 11:50:51,903 Test: data/test.txt
2024-03-09 11:50:52,133 No dev split found. Using 0% (i.e. 8 samples) of the train split as dev data


### Train Flair NER Model


In [5]:
sentence: Sentence = corpus.train[0]
for span in sentence.get_spans("ner"):
    print(span)

Span[0:3]: "gajanana marketing agencies" → VENDOR (1.0)
Span[15:16]: "29acspp1406b1zo" → VGSTIN (1.0)
Span[20:21]: "r.g.mart" → CUSTOMER (1.0)
Span[23:24]: "g-1868" → INVOICENO (1.0)
Span[25:26]: "10/10/2023" → INVOICEDATE (1.0)
Span[42:45]: "ugadi rbo tltr" → ITEMNAME (1.0)
Span[45:46]: "15159040" → HSN (1.0)
Span[47:48]: "80.00" → QUANTITY (1.0)
Span[48:49]: "pc" → UNIT (1.0)
Span[49:50]: "10400" → PRICE (1.0)
Span[50:51]: "8320.00" → AMOUNT (1.0)
Span[52:55]: "ugadi rboil 500m" → ITEMNAME (1.0)
Span[55:56]: "15159040" → HSN (1.0)
Span[57:58]: "40.00" → QUANTITY (1.0)
Span[58:59]: "pc" → UNIT (1.0)
Span[59:60]: "52.00" → PRICE (1.0)
Span[60:61]: "2080.00" → AMOUNT (1.0)
Span[64:65]: "10400.00" → GRANDTOTAL (1.0)
Span[76:77]: "9904.76" → TAXABLEAMT (1.0)


In [6]:
print(corpus.train[0].to_tagged_string('ner'))

Sentence[98]: "gajanana marketing agencies asm no 251-101-101-88 ward no-13block 10/6 tilak road sagar-577401 mob 9481281477 gstin 29acspp1406b1zo re tax invoice party r.g.mart invoice no g-1868 dated 10/10/2023 vehicle no gstin inv. type s.n description of goods/service hsn gst qty unit price amount 1 ugadi rbo tltr 15159040 5 80.00 pc 10400 8320.00 2. ugadi rboil 500m 15159040 5 40.00 pc 52.00 2080.00 grand total z 10400.00 hsn/sac tax rate taxable amt. cgst amt. sgst amt. 15159040 5 9904.76 247.62 247.62 rupees ten thousand four hundred only fssai license no-11221326000083 indian bank-ca a/c-7007807585 ifsc idiboo0s003 branch-sagar for gajanana marketing agencies" → ["gajanana marketing agencies"/VENDOR, "29acspp1406b1zo"/VGSTIN, "r.g.mart"/CUSTOMER, "g-1868"/INVOICENO, "10/10/2023"/INVOICEDATE, "ugadi rbo tltr"/ITEMNAME, "15159040"/HSN, "80.00"/QUANTITY, "pc"/UNIT, "10400"/PRICE, "8320.00"/AMOUNT, "ugadi rboil 500m"/ITEMNAME, "15159040"/HSN, "40.00"/QUANTITY, "pc"/UNIT, "52.00"/PRI

In [6]:
import flair, torch
flair.device = torch.device('cpu') 

In [2]:
from flair.embeddings import TransformerDocumentEmbeddings

# create an instance of WordEmbeddings
embeddings = TransformerDocumentEmbeddings(model='roberta-base',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

In [3]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

embedding_types = [
    WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

2024-03-09 11:52:20,908 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmpqgsekrux


100%|██████████| 69.7M/69.7M [00:09<00:00, 7.55MB/s]

2024-03-09 11:52:31,274 copying /tmp/tmpqgsekrux to cache at /home/dragon/.flair/embeddings/news-forward-0.4.1.pt
2024-03-09 11:52:31,311 removing temp file /tmp/tmpqgsekrux


2024-03-09 11:52:32,385 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpebbg5jd5


100%|██████████| 69.7M/69.7M [00:20<00:00, 3.60MB/s]

2024-03-09 11:52:53,362 copying /tmp/tmpebbg5jd5 to cache at /home/dragon/.flair/embeddings/news-backward-0.4.1.pt
2024-03-09 11:52:53,434 removing temp file /tmp/tmpebbg5jd5


In [4]:
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

label_type = 'ner'
label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)

# initialize sequence tagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=label_dict,
                                        tag_type=label_type
                                        )

# initialize trainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('data/flair-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2024-03-09 11:53:09,238 Computing label dictionary. Progress:


0it [00:00, ?it/s]
74it [00:00, 9336.94it/s]

2024-03-09 11:53:09,256 Dictionary created for label 'ner' with 14 values: ITEMNAME (seen 306 times), AMOUNT (seen 270 times), PRICE (seen 267 times), HSN (seen 260 times), QUANTITY (seen 209 times), UNIT (seen 191 times), VENDOR (seen 74 times), CUSTOMER (seen 74 times), VGSTIN (seen 73 times), GRANDTOTAL (seen 73 times), TAXABLEAMT (seen 73 times), INVOICEDATE (seen 71 times), INVOICENO (seen 70 times), CGSTIN (seen 58 times)
2024-03-09 11:53:09,257 SequenceTagger predicts: Dictionary with 57 tags: O, S-ITEMNAME, B-ITEMNAME, E-ITEMNAME, I-ITEMNAME, S-AMOUNT, B-AMOUNT, E-AMOUNT, I-AMOUNT, S-PRICE, B-PRICE, E-PRICE, I-PRICE, S-HSN, B-HSN, E-HSN, I-HSN, S-QUANTITY, B-QUANTITY, E-QUANTITY, I-QUANTITY, S-UNIT, B-UNIT, E-UNIT, I-UNIT, S-VENDOR, B-VENDOR, E-VENDOR, I-VENDOR, S-CUSTOMER, B-CUSTOMER, E-CUSTOMER, I-CUSTOMER, S-VGSTIN, B-VGSTIN, E-VGSTIN, I-VGSTIN, S-GRANDTOTAL, B-GRANDTOTAL, E-GRANDTOTAL, I-GRANDTOTAL, S-TAXABLEAMT, B-TAXABLEAMT, E-TAXABLEAMT, I-TAXABLEAMT, S-INVOICEDATE, B-IN

2024-03-09 11:53:09,456 ----------------------------------------------------------------------------------------------------
2024-03-09 11:53:09,457 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=59, bias=True)
  (loss_f

100%|██████████| 1/1 [00:02<00:00,  2.87s/it]

...

2024-03-09 12:07:38,767 DEV : loss 0.5056567192077637 - f1-score (micro avg)  0.3272
2024-03-09 12:07:38,773  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [9.765625e-05]
2024-03-09 12:07:38,774 ----------------------------------------------------------------------------------------------------
2024-03-09 12:07:38,774 learning rate too small - quitting training!
2024-03-09 12:07:38,775 ----------------------------------------------------------------------------------------------------
2024-03-09 12:07:38,776 Saving model ...


2024-03-09 12:07:39,818 Done.
2024-03-09 12:07:39,819 ----------------------------------------------------------------------------------------------------
2024-03-09 12:07:39,820 Loading model from best epoch ...
2024-03-09 12:07:40,714 SequenceTagger predicts: Dictionary with 59 tags: O, S-ITEMNAME, B-ITEMNAME, E-ITEMNAME, I-ITEMNAME, S-AMOUNT, B-AMOUNT, E-AMOUNT, I-AMOUNT, S-PRICE, B-PRICE, E-PRICE, I-PRICE, S-HSN, B-HSN, E-HSN, I-HSN, S-QUANTITY, B-QUANTITY, E-QUANTITY, I-QUANTITY, S-UNIT, B-UNIT, E-UNIT, I-UNIT, S-VENDOR, B-VENDOR, E-VENDOR, I-VENDOR, S-CUSTOMER, B-CUSTOMER, E-CUSTOMER, I-CUSTOMER, S-VGSTIN, B-VGSTIN, E-VGSTIN, I-VGSTIN, S-GRANDTOTAL, B-GRANDTOTAL, E-GRANDTOTAL, I-GRANDTOTAL, S-TAXABLEAMT, B-TAXABLEAMT, E-TAXABLEAMT, I-TAXABLEAMT, S-INVOICEDATE, B-INVOICEDATE, E-INVOICEDATE, I-INVOICEDATE, S-INVOICENO


100%|██████████| 1/1 [00:03<00:00,  3.49s/it]

2024-03-09 12:07:44,531 
Results:
- F-score (micro) 0.2393
- F-score (macro) 0.12
- Accuracy 0.1423

By class:
              precision    recall  f1-score   support

    ITEMNAME     0.2571    0.3214    0.2857        28
         HSN     0.5000    0.7727    0.6071        22
       PRICE     0.2174    0.2500    0.2326        20
    QUANTITY     0.1538    0.0952    0.1176        21
      AMOUNT     0.1429    0.0435    0.0667        23
        UNIT     0.3333    0.4167    0.3704        12
  TAXABLEAMT     0.0000    0.0000    0.0000         8
   INVOICENO     0.0000    0.0000    0.0000        10
  GRANDTOTAL     0.0000    0.0000    0.0000         8
 INVOICEDATE     0.0000    0.0000    0.0000         9
      VENDOR     0.0000    0.0000    0.0000         9
    CUSTOMER     0.0000    0.0000    0.0000         8
      CGSTIN     0.0000    0.0000    0.0000         8
      VGSTIN     0.0000    0.0000    0.0000         7

   micro avg     0.2932    0.2021    0.2393       193
   macro avg     0.1146

{'test_score': 0.2392638036809816}

In [27]:
print(tagger)

SequenceTagger(
  (embeddings): WordEmbeddings(
    'glove'
    (embedding): Embedding(400001, 100)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=29, bias=True)
  (loss_function): CrossEntropyLoss()
)


### Load Trained Model

In [28]:
from flair.data import Sentence

# load the trained model
model = SequenceTagger.load('data/flair-ner/final-model.pt')

# create a sentence
sentence = Sentence('tax invoice original for recipient invoice number invoice date haridhvarna agro llp 0064. 01/07/2023 spl plot no 3 industrial estate anavasi road sirsi ka - karnataka 581401 due date po number haridhvarna jyobile 9916706444 01/07/2023 9001 email haridhvarnaagro gmail.com gstin 29aapfh9297m1z4 po.nate 30/06/2023 fssai 21223172000136 rawgrnules private limited no 9 kerkai sagar - 577421 place of supply ka - karnataka mobile 1916362104056 gstin. 29aalcro358l1zu gstin 29aalcro358l1 dia vita mixed vegetable greens powder total amount in words taxable amount inr nineteen thousand nine hundred fifty rupees only eesrvaue terms and conditions if you have any questions about this invoice sgst value please contact haridhvarna agro llp. ph no 6362104056 sub-total net value goods once sold will not be taken back or exchanged subject to sirsi jurisdiction total amount. thank you for. your business received amount rawgrnules private limited no 9 kerkai sagar - 577421 mobile 1916362104056 zu 19000.00 475.00 2 475.00 19950.00 19950.00 0.00 fh balance amount 19950.00 t t imc i cgst sgst heniged i ipiable value i rate artouat rate anigune total tak amount oma ll as 000.00 2.5 475.00 2.5 2475.00 950.00 aad ot ct 4 ars00 e500 2950.00 upi id a i bank details for haridhvarna agro llp mabhatk oom name vitaminute hl ifsc code cnrbo010303 lie any uphapp wed account no 120000412812 bk due iii oy jpautin bank branch canara bank devikeri i a . road sirsi free gst billing and stock management using app authorised signature page of')

# predict the tags
model.predict(sentence)

# print the sentence with predicted tags
print(sentence.to_tagged_string())

2024-03-08 19:17:17,981 SequenceTagger predicts: Dictionary with 29 tags: O, B-ITEMNAME, I-ITEMNAME, B-AMOUNT, I-AMOUNT, B-PRICE, I-PRICE, B-HSN, I-HSN, B-QUANTITY, I-QUANTITY, B-UNIT, I-UNIT, B-VENDOR, I-VENDOR, B-CUSTOMER, I-CUSTOMER, B-VGSTIN, I-VGSTIN, B-INVOICEDATE, I-INVOICEDATE, B-INVOICENO, I-INVOICENO, B-GRANDTOTAL, I-GRANDTOTAL, B-TAXABLEAMT, I-TAXABLEAMT, B-CGSTIN, I-CGSTIN
Sentence[260]: "tax invoice original for recipient invoice number invoice date haridhvarna agro llp 0064. 01/07/2023 spl plot no 3 industrial estate anavasi road sirsi ka - karnataka 581401 due date po number haridhvarna jyobile 9916706444 01/07/2023 9001 email haridhvarnaagro gmail.com gstin 29aapfh9297m1z4 po.nate 30/06/2023 fssai 21223172000136 rawgrnules private limited no 9 kerkai sagar - 577421 place of supply ka - karnataka mobile 1916362104056 gstin. 29aalcro358l1zu gstin 29aalcro358l1 dia vita mixed vegetable greens powder total amount in words taxable amount inr nineteen thousand nine hundred fi

In [ ]:
from collections import defaultdict

def calculate_metrics(spacy_output, validation_data):
    TP = defaultdict(int)
    FP = defaultdict(int)
    FN = defaultdict(int)

    for doc, annotation in zip(spacy_output, validation_data):
        spacy_entities = set((ent.start_char, ent.end_char, ent.label_) for ent in doc.ents)
        test_entities = set((start, end, label) for start, end, label in annotation[1]['entities'])

        for entity in spacy_entities:
            if entity in test_entities:
                TP[entity[2]] += 1
            else:
                FP[entity[2]] += 1

        for entity in test_entities:
            if entity not in spacy_entities:
                FN[entity[2]] += 1

    metrics = {}
    for label in classes:
        precision = TP[label] / (TP[label] + FP[label]) if TP[label] + FP[label] != 0 else 0
        recall = TP[label] / (TP[label] + FN[label]) if TP[label] + FN[label] != 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

        metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

    return metrics

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_confusion_matrix(spacy_output, validation_data):
    y_true = []
    y_pred = []

    for doc, annotation in zip(spacy_output, validation_data):
        spacy_entities = set((ent.start_char, ent.end_char, ent.label_) for ent in doc.ents)
        test_entities = set((start, end, label) for start, end, label in annotation[1]['entities'])

        for entity in test_entities:
            y_true.append(entity[2])
            if entity in spacy_entities:
                y_pred.append(entity[2])
            else:
                y_pred.append('None')

    labels = list(set(y_true + y_pred))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
model_output = []

for data in test_data:
    output = nlp_ner(data[0])
    model_output.append(output)

metrics = calculate_metrics(model_output, test_data)
calculate_confusion_matrix(model_output, test_data)
print(metrics)

In [ ]:
def calculate_overall_metrics(metrics):
    micro_sum = defaultdict(int)
    macro_sum = defaultdict(float)

    for label, values in metrics.items():
        for metric, value in values.items():
            micro_sum[metric] += value
            macro_sum[metric] += value

    num_classes = len(metrics)
    micro_avg = {metric: value / num_classes for metric, value in micro_sum.items()}
    macro_avg = {metric: value / num_classes for metric, value in macro_sum.items()}

    return micro_avg, macro_avg

In [ ]:
micro_avg, macro_avg = calculate_overall_metrics(metrics)
print(micro_avg)
print(macro_avg)

In [ ]:
from google.colab import files
!zip -r /content/ner.zip /content/ner
files.download("/content/ner.zip")